In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import brentq
from scipy.stats import norm
from datetime import datetime, timedelta,date
import requests
import json
from polygon import RESTClient

## Historical Price of Underlying Asset

In [25]:
symbol = "NVDA"
ticker = yf.Ticker(symbol)
history = ticker.history(start="2024-01-01")
df = pd.DataFrame(history)
df.reset_index(inplace=True)
df['Date'] = df['Date'].dt.date  #
df.to_csv(f"DataSet/underlying.csv", index=False)


## Trade every first Tuesday, get the list of trade dates

In [ ]:
start_date = date(2024, 1, 1)
end_date = date(2025, 11, 30)

trade_dates = []
trade_dates_str = []
current = date(start_date.year, start_date.month, 1)

while current <= end_date:
    days_to_tuesday = (1 - current.weekday()) % 7
    first_tuesday = current.replace(day=1) + timedelta(days=days_to_tuesday)
    if start_date <= first_tuesday <= end_date:
        trade_dates.append(first_tuesday)
        trade_dates_str.append(first_tuesday.strftime("%Y-%m-%d"))
    if current.month == 12:
        current = date(current.year + 1, 1, 1)
    else:
        current = date(current.year, current.month + 1, 1)

# for d, ds in zip(trade_dates, trade_dates_str):
#     print(repr(d), ds)

## Get the underlying Price at each trading dates

In [ ]:
s0_list = []
underlying_df = pd.read_csv("DataSet/underlying.csv", parse_dates=['Date'])
underlying_df.set_index('Date', inplace=True)

for date_str in trade_dates_str:
    date_pd = pd.to_datetime(date_str)
    close_price = underlying_df.loc[date_pd]['Close']
    s0_list.append(close_price)
# print(s0_list)

In [ ]:
def atm_option(symbol, date):




for idx in range(len(s0_list)):
    date = trade_dates[idx]
    date_str = trade_dates_str[idx]
    s0 = s0_list[idx]
    atm_call, atm_put = atm_option(symbol, s0, date)
    
